In [16]:
# Milan Kovacs - 21308128, Caoimhe Cahill - 21331308
# The code runs to execution

In [17]:
# Imports
import numpy as np

In [32]:
# Constructor
class MultiLayerPerceptron:
    def __init__(self, layers: list , training_data: list, learning_rate: float = 0.2, epoch: int = 1_000,  k_fold: int = 6) -> None:
        # Declarations
        self.layers  = layers
        self.num_layers = len(self.layers)
        self.weights = []
        self.biases  = []
        self.learning_rate = learning_rate
        self.epoch = epoch
        self.training_data = training_data
        self.testing_data  = []
        self.k_fold = k_fold
        
        # Setting up
        self.set_weights()
        self.set_biases()
        
        # Transform data
        
        # Completion
        print(str(self))
    
    """
        * Generates a list of array shapes for each weight that corresponds to each weight in a given layer.
        * for example: if our layer is [4, 5, 2, 1], then we will have two lists to zip through
        * [4, 5, 2] and [5, 2, 1] which means on first iteration, we have our input layer of [4] nodes connecting to each [5] nodes 
        * in the first hidden layer. This will generate a shape of 5 lists, with 4 weights each.
        * Each shape corresponds to a node within our hidden layer (of 5 nodes) and in each shape the index 
        * corresponds to the index of our input layer. This is for the rest of the layers. 
    """
    def set_weights(self) -> None:
        layers_after_input_layer = self.layers[1:]
        layers_before_output_layer = self.layers[:-1]
        self.weights = [
            np.random.randn(y, x)
            for x, y in zip(layers_before_output_layer, layers_after_input_layer)
        ]
    
    """
        * Generates a list of Y by 1 dimensional list, where Y is the given layer size.
        * This means we can correspond each index to a layer of biases,
        * and within it, we can correspond each bias to a given node inside of that layer
    """
    def set_biases(self) -> None:
        layers_after_input = self.layers[1:]
        self.biases = [
            np.random.randn(y, 1) 
            for y in layers_after_input
        ]
        
    def __str__(self) -> str:
        return "Network generated with the following layers: " + str(self.layers)


sample = MultiLayerPerceptron([4, 5, 2, 1])


Network generated with the following layers: [4, 5, 2, 1]


In [19]:
# Pass forward

In [20]:
# Training

In [28]:
# K-Fold for training and testing data
print(sample.weights)

[array([[ 0.59428402,  0.94397662, -0.51891439,  0.18960908],
       [-0.40973078, -1.41240237,  1.54990842,  1.38777465],
       [ 0.16187396, -1.77100826, -0.78233533,  0.57267828],
       [ 0.79979405,  0.82078084, -2.0903556 ,  1.6304427 ],
       [-1.52159401,  0.41786871, -1.78633   , -1.87557606]]), array([[-1.15948619,  0.90332813,  0.75834005, -0.34528691,  0.23733199],
       [-0.06189222,  0.52659231,  0.09288681,  2.47968626, -0.57923084]]), array([[0.09951519, 0.63212274]])]
